# Calculate the correlations for each layer and create input RDMs

## Set up the environment

In [ ]:
import numpy as np
import os
import json
import matplotlib.pyplot as plt
import seaborn
import pandas as pd
import scipy 

## Select the model and layer

In [ ]:
model_type = 'b' #b, bl, b_d
model_name='{}_net_scenes'.format(model_type)
path_dnn_activ = '/scratch/agnek95/PDM/DATA/'

#define variables
NR_OF_SAMPLES = 60
if model_type=='b' or model_type=='bl':
    num_layers=7
elif model_type=='b_d':
    num_layers=14

#define layer names    
layer_names = []
if model_type=='b' or model_type=='b_d': #feedforward
    for layer_idx in range(num_layers):
        layer = 'ReLU_Layer_{}'.format(layer_idx)
        layer_names.append(layer)
elif model_type=='b_l': #rnn
    num_timepoints = 8
    for layer_idx in range(num_layers):
        for timepoint in range(num_timepoints):       
            layer_time = 'ReLU_Layer_{}_Time_{}'.format(layer_idx,timepoint)
            layer_names.append(layer_time)

#define path name
if model_type=='b':
    path_name='B'
    path_rdm='B_net'
elif model_type=='b_d':
    path_name='B_D'
    path_rdm='B_D_net'
elif model_type=='bl':
    path_name='RNN'
    path_rdm='02.11_2_rnn'

## Define the correlation function (correlation distance (Pearson) or Euclidean)

In [ ]:
def correlationd_matrix_pearson():
    file_name = os.path.join(path_dnn_activ+'{}_ACTIVATIONS/activations/'.format(path_name),'{}_activations.npy'.format(layer_name)) 
    act = np.load(file_name,mmap_mode='r')
    print(np.argwhere(np.isnan(act)))
    print(act.shape)
    correlationd = 1-np.corrcoef(act)                                
    return(correlationd)

In [ ]:
def correlationd_matrix_euclid(): 
    file_name = os.path.join(path_dnn_activ+'{}_ACTIVATIONS/activations/','{}_activations.npy'.format(path_name,layer_name)) 
    act = np.load(file_name,mmap_mode='r')
    print(np.argwhere(np.isnan(act)))
    print(act.shape)
    correlationd = np.zeros((NR_OF_SAMPLES,NR_OF_SAMPLES))
    correlationd[:] = np.nan
    
    for i in range(NR_OF_SAMPLES):
        for j in range(NR_OF_SAMPLES):  
            correlationd[i,j] = scipy.spatial.distance.euclidean(act[i,:],act[j,:])
            # print(correlationd[i,j])                                 
    return(correlationd)

# Create RDM

In [ ]:
corr_type='pearson'

for layer_name in layer_names:
    print('Calculating the correlations for model: ',model_name,'and layer: ',layer_name)
    if corr_type=='pearson':
        corr_matrix = correlationd_matrix_pearson()
    elif corr_type=='euclid':
        corr_matrix = correlationd_matrix_euclid()
    save_corr_path = os.path.join('/home/agnek95/SMST/PDM_FULL_EXPERIMENT/RESULTS_AVG/{}/Input_RDM_{}'.format(path_rdm,corr_type), '{}_Input_RDM_{}.npy'.format(layer_name,corr_type))
    print("Save Input RDM -> {}".format(save_corr_path))
    np.save(save_corr_path, np.array(corr_matrix)) 


# To plot/visualize RDM (Example)

In [ ]:
filename_rdm='/scratch/agnek95/PDM/DATA/RNN_ACTIVATIONS/activations/ReLU_Layer_0_Time_0_activations.npy'
rdm_7_8 = np.load(filename_rdm,mmap_mode='r')
fig = plt.figure(figsize=(17,13))
ax = seaborn.heatmap(rdm_7_8, cmap='rainbow')

## In case the load function does not work

In [ ]:
# #If the load function gives an error, do this
# np_load_old = np.load # modify the default parameters of np.load
# np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
# activations_shape = np.load(path)
# np.load = np_load_old